In [1]:
import numpy as np
import pandas as pd

In [2]:
tags = pd.read_csv('../input/stacksample/Tags.csv')

In [3]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [4]:
temp = tags['Tag'].value_counts()

In [5]:
temp = pd.DataFrame(temp)
temp = temp.reset_index()
temp.head()

,index,Tag
0,javascript,124155
1,java,115212
2,c#,101186
3,php,98808
4,android,90659


In [6]:
temp.rename(columns = {'index':'Tag' , 'Tag':'Counts'}, inplace = True)
temp.head()

,Tag,Counts
0,javascript,124155
1,java,115212
2,c#,101186
3,php,98808
4,android,90659


In [7]:
tags = pd.merge(tags , temp , on = 'Tag')

In [8]:
tags.head()

,Id,Tag,Counts
0,80,flex,2430
1,10870,flex,2430
2,14350,flex,2430
3,20510,flex,2430
4,20910,flex,2430


## **Choosing the 20 immediate tags which have count >= 4021 - for a decent data size and less imbalance in classes i.e., tags**

In [9]:
temp.loc[temp["Counts"] >= 4021, :].index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             97,  98,  99, 100, 101, 102, 103, 104, 105, 106],
           dtype='int64', length=107)

In [10]:
# stores the row indices of the chosen tags (i.e., tags with counts >= 4021) from the table - temp
chosen_tags_indices = list(range(106, -1, -1))[:20]
chosen_tags_names = list(temp.loc[chosen_tags_indices, "Tag"].values)

In [11]:
#tags.where(tags['Counts']>=20345 , inplace = True)
tags = tags.loc[tags["Tag"].isin(chosen_tags_names), :]

In [12]:
tags.head()

,Id,Tag,Counts
2430,80,actionscript-3,4039
2431,10870,actionscript-3,4039
2432,14350,actionscript-3,4039
2433,33590,actionscript-3,4039
2434,63390,actionscript-3,4039


In [13]:
tags = tags.sort_values(by = 'Id')

In [14]:
tags.head()

,Id,Tag,Counts
2430,80,actionscript-3,4039
263927,330,class,4485
325131,930,loops,4121
343735,1760,unit-testing,4437
343736,3150,unit-testing,4437


In [15]:
tags['Id'] = tags['Id'].astype('int32')

In [16]:
tags.shape

(86225, 3)

### **Tags file ready**

## **Importing the Questions file -**

In [17]:
q = pd.read_csv('../input/stacksample/Questions.csv' , encoding='latin-1')

In [18]:
q.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [19]:
df = q.drop(['OwnerUserId' ,  'CreationDate' , 'ClosedDate' , 'Score'] , axis =1)

In [20]:
def comb_tags_func(x):
    list_tags = x.values
    tag_comb = ' '.join(list_tags)
    return tag_comb

In [21]:
mapping_dict = tags.groupby('Id')['Tag'].apply(comb_tags_func)

In [22]:
df['tag'] = df['Id'].map(mapping_dict)

In [23]:
df.head()

,Id,Title,Body,tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,actionscript-3
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,NaN
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,NaN
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,NaN


In [24]:
df.dropna(inplace=True)

In [25]:
df['tag'] = df['tag'].apply(lambda x : x.split())

In [26]:
df.head()

,Id,Title,Body,tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,[actionscript-3]
5,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[class]
10,930,How do I connect to a database and loop over a...,<p>What's the simplest way to connect and quer...,[loops]
20,1760,.NET Unit Testing packages?,<p>Getting back into a bit more .NET after a f...,[unit-testing]
36,3150,How to set up unit testing for Visual Studio C++,<p>I'm having trouble figuring out how to get ...,[unit-testing]


In [27]:
df['Body'].iloc[1]

"<p>I am working on a collection of classes used for video playback and recording. I have one main class which acts like the public interface, with methods like <code>play()</code>, <code>stop()</code>, <code>pause()</code>, <code>record()</code> etc... Then I have workhorse classes which do the video decoding and video encoding. </p>\n\n<p>I just learned about the existence of nested classes in C++, and I'm curious to know what programmers think about using them. I am a little wary and not really sure what the benefits/drawbacks are, but they seem (according to the book I'm reading) to be used in cases such as mine.</p>\n\n<p>The book suggests that in a scenario like mine, a good solution would be to nest the workhorse classes inside the interface class, so there are no separate files for classes the client is not meant to use, and to avoid any possible naming conflicts? I don't know about these justifications. Nested classes are a new concept to me. Just want to see what programmers 

## **Text cleaning and preprocessing -**

In [28]:
df['Body']=df['Body'].str.replace('<p>',' ')

In [29]:
import re
df['Body']=df['Body'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x))

In [30]:
df['Body']=df['Body'].str.replace('</p>',' ')
df['Body']=df['Body'].str.replace('\n',' ')
df['Body']=df['Body'].str.replace('</a>',' ')

In [31]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

stemmer = SnowballStemmer('english')
stop_words=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df['Body']=df['Body'].apply(lambda x : x.lower())
df['Body'] = df['Body'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

df['Body']=df['Body'].str.replace('<a href=" ">','')

df['Body'] = df['Body'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3]))

In [33]:
df['Body'] = [stemmer.stem(i) for i in df['Body']]

In [34]:
df['Title'][0]

'SQLStatement.execute() - multiple queries in one statement'

In [35]:
df['Title'] = [i.replace("?", "") for i in df['Title']]
df['Title'] = df['Title'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))
df['Title'] = [stemmer.stem(i) for i in df['Title']]

## **Model building begins -**

In [36]:
df.head()

,Id,Title,Body,tag
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3]
5,330,should i use nested classes cas,working collection classes used video playback...,[class]
10,930,how i connect database loop recordset c#,what's simplest connect query database record,[loops]
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing]
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing]


In [37]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

s=df['tag']
tag_dummies=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index).add_prefix('tag_')
df=pd.concat([df,tag_dummies],axis=1)

In [38]:
df.head()

,Id,Title,Body,tag,tag_actionscript-3,tag_android-layout,tag_class,tag_date,tag_email,tag_google-chrome,...,tag_python-3.x,tag_sockets,tag_sorting,tag_symfony2,tag_tsql,tag_unit-testing,tag_validation,tag_visual-studio-2010,tag_wcf,tag_xaml
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,330,should i use nested classes cas,working collection classes used video playback...,[class],0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,930,how i connect database loop recordset c#,what's simplest connect query database record,[loops],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing],0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing],0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [39]:
pred_features=[feature for feature in df.columns if 'tag_' in feature]
pred_features

['tag_actionscript-3',
 'tag_android-layout',
 'tag_class',
 'tag_date',
 'tag_email',
 'tag_google-chrome',
 'tag_google-maps',
 'tag_jsp',
 'tag_loops',
 'tag_maven',
 'tag_python-3.x',
 'tag_sockets',
 'tag_sorting',
 'tag_symfony2',
 'tag_tsql',
 'tag_unit-testing',
 'tag_validation',
 'tag_visual-studio-2010',
 'tag_wcf',
 'tag_xaml']

In [40]:
y = df[pred_features]

In [41]:
y.shape

(84884, 20)

In [42]:
df['text'] = df['Body'] + ' ' + df['Title']
df['num_tags'] = df.loc[:, "tag_actionscript-3":"tag_xaml"].sum(axis=1)
df.head()

,Id,Title,Body,tag,tag_actionscript-3,tag_android-layout,tag_class,tag_date,tag_email,tag_google-chrome,...,tag_sorting,tag_symfony2,tag_tsql,tag_unit-testing,tag_validation,tag_visual-studio-2010,tag_wcf,tag_xaml,text,num_tags
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,i've written database generation script want e...,1
5,330,should i use nested classes cas,working collection classes used video playback...,[class],0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,working collection classes used video playback...,1
10,930,how i connect database loop recordset c#,what's simplest connect query database record,[loops],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,what's simplest connect query database record ...,1
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing],0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"getting back .net few-years using full-time, w...",1
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing],0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,trouble figuring testing framework usable visu...,1


In [43]:
df['num_tags'].value_counts()

1    83554
2     1319
3       11
Name: num_tags, dtype: int64

### **Maintaining the ratio of the number of tags(for each question) in each fold -**

In [44]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split

In [45]:
df['kfold'] = -1
df = df.sample(frac = 1, random_state = 100).reset_index(drop = True)

In [46]:
skf = model_selection.StratifiedKFold(n_splits = 5)

# Filling the new KFold column - 
for f , (t_ , v_) in enumerate(skf.split(X = df , y = df.num_tags.values)) :
    df.loc[v_ , 'kfold'] = f

In [47]:
df.kfold.value_counts()

0    16977
1    16977
2    16977
3    16977
4    16976
Name: kfold, dtype: int64

In [49]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english' , max_features = 2500)

In [50]:
X = df['text']
y = df[pred_features]

In [56]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2)

In [57]:
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [58]:
y_train.shape

(67907, 20)

In [59]:
std_obj = StandardScaler()

In [ ]:
#sgd_model = SGDClassifier(loss='hinge', alpha=0.00001, penalty='elasticnet', l1_ratio = 0.70, max_iter = 1000, tol = 1e-3, learning_rate = 'adaptive', 
                          #early_stopping = True, n_iter_no_change = 3, class_weight = None, eta0 = 0.1)
#multilabel_model = MultiOutputClassifier(sgd_model, n_jobs=-1)

#multilabel_model = MultiOutputClassifier(model1, n_jobs = -1)
#multilabel_model = BinaryRelevance(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'))
#multilabel_model = ClassifierChain(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'))
#multilabel_model = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
#multilabel_model = MultiOutputClassifier(DecisionTreeClassifier(max_depth = 11, class_weight = 'balanced'), n_jobs=-1)
#multilabel_model = MultiOutputClassifier(lr, n_jobs = -1)

In [60]:
multilabel_model = MultiOutputClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs = -1)

In [61]:
fold_num = 1
#log_reg_coefs = []
#log_reg_intercepts = []
train_acc_scores = []
val_acc_scores = []
train_ham_loss = []
val_ham_loss = []
for kfold_idx in df['kfold'].value_counts().index:
    print(f"Fold num : {fold_num}", end = "\n")
    X_train = df[df['kfold'] != kfold_idx].loc[:, 'text'].values
    X_train = tfidf_vectorizer.fit_transform(X_train)
    y_train = df[df['kfold'] != kfold_idx].loc[:, "tag_actionscript-3":"tag_xaml"].values
    X_val = df[df['kfold'] == kfold_idx].loc[:, 'text'].values
    X_val = tfidf_vectorizer.transform(X_val)
    y_val = df[df['kfold'] == kfold_idx].loc[:, "tag_actionscript-3":"tag_xaml"].values
    
    
    multilabel_model.fit(X_train , y_train)
    preds_train = multilabel_model.predict(X_train)
    preds_val = multilabel_model.predict(X_val)
    
    train_acc_scores.append(accuracy_score(y_true = y_train, y_pred = preds_train))
    val_acc_scores.append(accuracy_score(y_true = y_val, y_pred = preds_val))
    train_ham_loss.append(hamming_loss(y_true = y_train, y_pred = preds_train))
    val_ham_loss.append(hamming_loss(y_true = y_val, y_pred = preds_val))
    
    print(f"Train Accuracy : {train_acc_scores[-1]}")
    print(f"Val Accuracy : {val_acc_scores[-1]}")
    
    print(f"Train Hamming Loss : {train_ham_loss[-1]}")
    print(f"Val Hamming Loss : {val_ham_loss[-1]}")
    
    print("\n")
    fold_num = fold_num + 1
print(f"Mean Train Accuracy : {np.mean(train_acc_scores)}")
print(f"Mean Val Accuracy : {np.mean(val_acc_scores)}")

print(f"Mean Train Hamming Loss : {np.mean(train_ham_loss)}")
print(f"Mean Val Hamming Loss : {np.mean(val_ham_loss)}")

Fold num : 1
Train Accuracy : 0.8045562313163591
Val Accuracy : 0.783942981681098
Train Hamming Loss : 0.011338300911540783
Val Hamming Loss : 0.012687754020144902


Fold num : 2
Train Accuracy : 0.803245615326844
Val Accuracy : 0.7758732402662426
Train Hamming Loss : 0.011334619405952259
Val Hamming Loss : 0.013232608823702656


Fold num : 3
Train Accuracy : 0.804232258824569
Val Accuracy : 0.7746362726041115
Train Hamming Loss : 0.01135376323501259
Val Hamming Loss : 0.013173705601696412


Fold num : 4
Train Accuracy : 0.8031278071480112
Val Accuracy : 0.7754609177121988
Train Hamming Loss : 0.011408985818840473
Val Hamming Loss : 0.013229663662602344


Fold num : 5
Train Accuracy : 0.8054721093243801
Val Accuracy : 0.7848727615457116
Train Hamming Loss : 0.011299110561347706
Val Hamming Loss : 0.012620758718190387


Mean Train Accuracy : 0.8041268043880325
Mean Val Accuracy : 0.7789572347618725
Mean Train Hamming Loss : 0.011346955986538763
Mean Val Hamming Loss : 0.0129888981652673

### **OneVsRest and MultiOutput almost give the same results**

## **Training and evaluating the best model -**

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, 'text'], df.loc[:, "tag_actionscript-3":"tag_xaml"], test_size = 0.2, random_state = 25)

In [63]:
# Transforming the X variables before modelling - 
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [64]:
multilabel_model = MultiOutputClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
multilabel_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=1e-05, loss='log',
                                              penalty='l1'),
                      n_jobs=-1)

In [65]:
print("Train acc. score:", accuracy_score(y_true = y_train, y_pred = multilabel_model.predict(X_train)), end = "\n")
print("Train hamming loss:", hamming_loss(y_true = y_train, y_pred = multilabel_model.predict(X_train)), end = "\n\n")
print("Test acc. score:", accuracy_score(y_true = y_test, y_pred = multilabel_model.predict(X_test)), end = "\n")
print("Test hamming loss:", hamming_loss(y_true = y_test, y_pred = multilabel_model.predict(X_test)), end = "\n\n")

Train acc. score: 0.8013165063984568
Train hamming loss: 0.011484088532846394

Test acc. score: 0.7789362078105673
Test hamming loss: 0.01294103787477175



## **Creating a function for the model -**

In [66]:
def model_func(question):
    # processing the question - 
    question = question.replace('<p>',' ')
    
    f1 = lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)
    question = f1(question)
    
    question = question.replace('</p>',' ')
    question = question.replace('\n',' ')
    question = question.replace('</a>',' ')
    
    f2 = lambda x : x.lower()
    question = f2(question)
    
    f3 = lambda x : ' '.join([w for w in x.split() if not w in stop_words])
    question = f3(question)
    
    question = question.replace('<a href=" ">','')
    
    f4 = lambda x : ' '.join([w for w in x.split() if len(w)>3])
    question = f4(question)
    
    # applying tfidf - 
    tr_question = tfidf_vectorizer.transform(pd.Series([question]))
    
    # predicting the tags for the input question - 
    return multilabel_model.predict(tr_question)

In [67]:
sample_input = "I'm not able to load tsql in visual-studio and also in chrome , I'm such a noob :("
model_func(sample_input)

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [68]:
sample_output = model_func(sample_input)

In [69]:
# displaying the tags for the sample input - 
mlb.inverse_transform(sample_output)

[('google-chrome', 'tsql')]

### **Saving the model and preprocessing objects in pickle files for the web app -**

In [ ]:
import pickle
pickle.dump(multilabel_model, open("sof_first_model.sav", 'wb'))

In [ ]:
pickle.dump(tfidf_vectorizer, open("tfidf_object.pickle", "wb"))

In [ ]:
pickle.dump(mlb, open("multilabel_binarizer.pickle", "wb"))